In [52]:
!pip install seaborn matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os

from transformers import TFAutoModelForSequenceClassification

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [57]:
df_init = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/wagon/books_processed_top60.csv', engine="python")
df_init

,Author,Book
0,Twain Mark,﻿The Project Gutenberg EBook of Chapters from ...
1,Ebers Georg,"﻿The Project Gutenberg EBook Margery, by Georg..."
2,Motley John Lothrop,﻿The Project Gutenberg EBook The Life of John ...
3,Warner Charles Dudley,"﻿The Project Gutenberg EBook of England, by Ch..."
4,Ebers Georg,﻿The Project Gutenberg EBook In The Fire Of Th...
...,...,...
959,Warner Charles Dudley,"﻿The Project Gutenberg EBook of As We Go, by C..."
960,Ebers Georg,﻿The Project Gutenberg EBook A Word Only A Wor...
961,Montaigne Michel de,﻿The Project Gutenberg Etext of Quotations fro...
962,Motley John Lothrop,﻿The Project Gutenberg EBook of The Life of Jo...


In [58]:
def strip_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)
def clean_special_chars(text):
    clean = re.sub(r'[\n\r\t]', '', text)
    return clean
df_init['Book'] = df_init['Book'].apply(strip_html)
df_init['Book'] = df_init['Book'].apply(clean_special_chars)
df_init

,Author,Book
0,Twain Mark,﻿The Project Gutenberg EBook of Chapters from ...
1,Ebers Georg,"﻿The Project Gutenberg EBook Margery, by Georg..."
2,Motley John Lothrop,﻿The Project Gutenberg EBook The Life of John ...
3,Warner Charles Dudley,"﻿The Project Gutenberg EBook of England, by Ch..."
4,Ebers Georg,﻿The Project Gutenberg EBook In The Fire Of Th...
...,...,...
959,Warner Charles Dudley,"﻿The Project Gutenberg EBook of As We Go, by C..."
960,Ebers Georg,﻿The Project Gutenberg EBook A Word Only A Wor...
961,Montaigne Michel de,﻿The Project Gutenberg Etext of Quotations fro...
962,Motley John Lothrop,﻿The Project Gutenberg EBook of The Life of Jo...


In [59]:
def chunkator(text, LEN=300):
    txt = text.split()
    chunks = [txt[i:i+LEN] for i in range(0, len(txt), LEN)]
    # chunked_text = ' '.join([' '.join(chunk) for chunk in chunks])
    return chunks

In [60]:
# Create an empty list to store the chunks
chunks_list = []

# Iterate over each row in the original DataFrame
for index, row in df_init.iterrows():
    author = row['Author']
    book = row['Book']

    # Split the book text into chunks of 512 characters
    chunks = chunkator(book)

    # Create a DataFrame with author and book chunks
    chunk_df = pd.DataFrame({'Author': author, 'Book': chunks})

    # Append the chunk DataFrame to the list
    chunks_list.append(chunk_df)

# Concatenate all the chunk DataFrames into a new DataFrame
new_df = pd.concat(chunks_list, ignore_index=True)

# Print the new DataFrame
new_df

,Author,Book
0,Twain Mark,"[﻿The, Project, Gutenberg, EBook, of, Chapters..."
1,Twain Mark,"[are, of, asort, which, he, is, familiar, with..."
2,Twain Mark,"[at, any, time, during, its, existence, who, w..."
3,Twain Mark,"[and, unchangingly, bitter, against, Charles,a..."
4,Twain Mark,"[same, difficulty., In, fact, he, looked, dist..."
...,...,...
292655,Shelley Percy Bysshe,"[refund, of, any, money, paid, by, a, user, wh..."
292656,Shelley Percy Bysshe,"[work, under, this, agreement,, disclaim, alll..."
292657,Shelley Percy Bysshe,"[invalidity, or, unenforceability, of, anyprov..."
292658,Shelley Percy Bysshe,"[U.S., federal, laws, and, your, state's, laws..."


In [61]:
# Iterate over each row in the DataFrame
for index, row in new_df.iterrows():
    # Convert the list of words into a string
    book_string = ' '.join(row['Book'])

    # Update the "Book" column with the string value
    new_df.at[index, 'Book'] = book_string

new_df

,Author,Book
0,Twain Mark,﻿The Project Gutenberg EBook of Chapters from ...
1,Twain Mark,are of asort which he is familiar with in his ...
2,Twain Mark,at any time during its existence who would be ...
3,Twain Mark,"and unchangingly bitter against Charles,and I ..."
4,Twain Mark,"same difficulty. In fact he looked distraught,..."
...,...,...
292655,Shelley Percy Bysshe,refund of any money paid by a user who notifie...
292656,Shelley Percy Bysshe,"work under this agreement, disclaim allliabili..."
292657,Shelley Percy Bysshe,invalidity or unenforceability of anyprovision...
292658,Shelley Percy Bysshe,U.S. federal laws and your state's laws.The Fo...


In [62]:
top40 = new_df['Author'].value_counts().head(40).index.tolist()
top40

['Motley John Lothrop',
 'Twain Mark',
 'Shakespeare William',
 'Ebers Georg',
 'Larned J. N. (Josephus Nelson)',
 'Zola Émile',
 'Dumas Alexandre',
 'Fishburne William Brett',
 'Grant Ulysses S. (Ulysses Simpson)',
 'Thomas Aquinas Saint',
 'Tolstoy Leo graf',
 'Steele Richard Sir',
 'Walsh James J. (James Joseph)',
 'Holmes Oliver Wendell',
 'Whittier John Greenleaf',
 'Warner Charles Dudley',
 'Plutarch',
 'Hardy Thomas',
 'Browning Robert',
 'Benton Thomas Hart',
 'Auerbach Berthold',
 'Wood J. G. (John George)',
 'Lincoln Abraham',
 'Rose J. Holland (John Holland)',
 'Haeckel Ernst',
 'Smith Adam',
 'Montaigne Michel de',
 'Shelley Percy Bysshe',
 'Mill John Stuart',
 'Whewell William',
 'Brown Goold',
 'Poe Edgar Allan',
 'Rose Joshua',
 'Josephus Flavius',
 'Lewis Meriwether',
 'Spenser Edmund',
 'Lemprière John',
 'Polo Marco',
 'Nuttall P. Austin',
 'Rymer James Malcolm']

In [63]:
new_df = new_df[new_df['Author'].isin(top40)]
new_df

,Author,Book
0,Twain Mark,﻿The Project Gutenberg EBook of Chapters from ...
1,Twain Mark,are of asort which he is familiar with in his ...
2,Twain Mark,at any time during its existence who would be ...
3,Twain Mark,"and unchangingly bitter against Charles,and I ..."
4,Twain Mark,"same difficulty. In fact he looked distraught,..."
...,...,...
292655,Shelley Percy Bysshe,refund of any money paid by a user who notifie...
292656,Shelley Percy Bysshe,"work under this agreement, disclaim allliabili..."
292657,Shelley Percy Bysshe,invalidity or unenforceability of anyprovision...
292658,Shelley Percy Bysshe,U.S. federal laws and your state's laws.The Fo...


In [64]:
# keep the same amount of chunks per author
amount_per_author = new_df['Author'].value_counts().min()
amount_per_author

2337

In [65]:
new_df = new_df.groupby('Author').head(amount_per_author)

In [66]:
new_df

,Author,Book
0,Twain Mark,﻿The Project Gutenberg EBook of Chapters from ...
1,Twain Mark,are of asort which he is familiar with in his ...
2,Twain Mark,at any time during its existence who would be ...
3,Twain Mark,"and unchangingly bitter against Charles,and I ..."
4,Twain Mark,"same difficulty. In fact he looked distraught,..."
...,...,...
273713,Poe Edgar Allan,lacerating the feet against the pavement was l...
273714,Poe Edgar Allan,"all the turrets around about Jerusalem, and th..."
273715,Poe Edgar Allan,and hast sojourned among them who dabble with ...
273716,Poe Edgar Allan,rope became indistinctly visible—“Booshoh he!”...


In [67]:
# get the labels
labels = new_df['Author'].tolist()

# convert labels to nparray
labels_text = np.array(labels)

# apply label encoding to labels
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(labels_text)
labels = np.array(labels)
labels

labels_clean = le.inverse_transform(labels)

new_df['Author_Encoded']=labels

<ipython-input-67-794b768605f6>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Author_Encoded']=labels


In [68]:
new_df

,Author,Book,Author_Encoded
0,Twain Mark,﻿The Project Gutenberg EBook of Chapters from ...,33
1,Twain Mark,are of asort which he is familiar with in his ...,33
2,Twain Mark,at any time during its existence who would be ...,33
3,Twain Mark,"and unchangingly bitter against Charles,and I ...",33
4,Twain Mark,"same difficulty. In fact he looked distraught,...",33
...,...,...,...
273713,Poe Edgar Allan,lacerating the feet against the pavement was l...,21
273714,Poe Edgar Allan,"all the turrets around about Jerusalem, and th...",21
273715,Poe Edgar Allan,and hast sojourned among them who dabble with ...,21
273716,Poe Edgar Allan,rope became indistinctly visible—“Booshoh he!”...,21


In [69]:
list_text = []
for _, row in new_df.iterrows():
    list_text.append(row["Book"])

list_labels =[]
for _, row in new_df.iterrows():
    list_labels.append(row["Author_Encoded"])

In [70]:
data_dict = {'text': list_text, 'labels': list_labels}

In [71]:
from datasets import Dataset
ds = Dataset.from_dict(data_dict)
ds

Dataset({
    features: ['text', 'labels'],
    num_rows: 93480
})

In [72]:
from datasets import DatasetDict

# 70% train, 30% test + validation
ds_train_testvalid = ds.train_test_split(test_size=0.3, shuffle=True)

# Split the 30% test + valid in half test, half valid
ds_testvalid = ds_train_testvalid['test'].train_test_split(test_size=0.5)

# gather everyone if you want to have a single DatasetDict
ds_split = DatasetDict({
    'train': ds_train_testvalid['train'],
    'test': ds_testvalid['test'],
    'valid': ds_testvalid['train']})

ds_split

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 65436
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 14022
    })
    valid: Dataset({
        features: ['text', 'labels'],
        num_rows: 14022
    })
})

In [73]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [74]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_ds_split = ds_split.map(preprocess_function, batched=True)

Map:   0%|          | 0/65436 [00:00<?, ? examples/s]

Map:   0%|          | 0/14022 [00:00<?, ? examples/s]

Map:   0%|          | 0/14022 [00:00<?, ? examples/s]

In [75]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [76]:
import evaluate

accuracy = evaluate.load("accuracy")

In [77]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [78]:
data_tmp = new_df[['Author', 'Author_Encoded']].groupby(['Author_Encoded']).min()

id2label = data_tmp.to_dict()['Author']

label2id = {v: k for k, v in id2label.items()}

In [79]:
import tensorflow as tf

In [80]:
from transformers import create_optimizer

batch_size = 32
num_epochs = 20
batches_per_epoch = len(tokenized_ds_split["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=1e-5, num_warmup_steps=0, num_train_steps=total_train_steps)
# optimizer = tf.keras.optimizers.legacy.Adam(3e-5)

#INIT & COMPILE MODE

In [81]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=40, id2label=id2label, label2id=label2id)

# Set the classifier layer to be trainable
model.classifier.trainable = True
# Make the BERT layers non-trainable
for layer in model.layers:
    if "bert" in layer.name:
        layer.trainable = False

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [82]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_ds_split["train"],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_ds_split["valid"],
    shuffle=False,
    batch_size=32,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_ds_split["test"],
    shuffle=False,
    batch_size=32,
    collate_fn=data_collator,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [83]:
tf_train_set, tf_validation_set

(<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(32, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(32, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(32,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [84]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [85]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  30760     
                                                                 
 dropout_59 (Dropout)        multiple                  0         
                                                                 
Total params: 66,984,232
Trainable params: 621,352
Non-trainable params: 66,362,880
_________________________________________________________________


#CALLBACKS

In [86]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(
    monitor="val_accuracy",
    patience=10,
    restore_best_weights=True,
    verbose=1)

from keras.callbacks import ModelCheckpoint

checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/wagon/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              save_weights_only=True,
                              verbose=1)

from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,
                              patience=3, min_lr=0.0000001)

#TRAIN MODEL

In [87]:
history = model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=20, callbacks=[es, cp_callback, reduce_lr], verbose=1)

Epoch 1/20
2044/2044 [==============================] - ETA: 0s - loss: 6.0322 - accuracy: 0.1200
Epoch 1: saving model to /content/drive/MyDrive/Colab Notebooks/wagon/cp.ckpt
2044/2044 [==============================] - 205s 97ms/step - loss: 6.0322 - accuracy: 0.1200 - val_loss: 3.9534 - val_accuracy: 0.3047 - lr: 9.5002e-06
Epoch 2/20
2044/2044 [==============================] - ETA: 0s - loss: 3.9122 - accuracy: 0.2512
Epoch 2: saving model to /content/drive/MyDrive/Colab Notebooks/wagon/cp.ckpt
2044/2044 [==============================] - 197s 96ms/step - loss: 3.9122 - accuracy: 0.2512 - val_loss: 3.1485 - val_accuracy: 0.4899 - lr: 9.0002e-06
Epoch 3/20
2044/2044 [==============================] - ETA: 0s - loss: 3.5179 - accuracy: 0.3528
Epoch 3: saving model to /content/drive/MyDrive/Colab Notebooks/wagon/cp.ckpt
2044/2044 [==============================] - 197s 96ms/step - loss: 3.5179 - accuracy: 0.3528 - val_loss: 2.8210 - val_accuracy: 0.5620 - lr: 8.5002e-06
Epoch 4/20
20

In [88]:
model.evaluate(tf_test_set, verbose=1)

439/439 [==============================] - 30s 69ms/step - loss: 2.6625 - accuracy: 0.6893


[2.662482976913452, 0.6893453001976013]

#SAVE MODEL

In [89]:
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_model6', save_format="tf")

In [90]:
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_tokenizer6')

('/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_tokenizer6/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_tokenizer6/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_tokenizer6/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_tokenizer6/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_tokenizer6/tokenizer.json')

#LOAD MODEL

In [109]:
from transformers import TextClassificationPipeline

predict_model = TFAutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_model6')
predict_tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_tokenizer6')

pipe = TextClassificationPipeline(model=predict_model, tokenizer=predict_tokenizer, return_all_scores=True)

# outputs a list of dicts like [[{'label': 'NEGATIVE', 'score': 0.0001223755971295759},  {'label': 'POSITIVE', 'score': 0.9998776316642761}]]
# pipe(text)

Some layers from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_model6 were not used when initializing TFDistilBertForSequenceClassification: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_model6 and are newly initialized: ['dropout_159']
You should probably TRAIN this model on a down-stream task to be able to use it for prediction

#TEXT TO PREDICT

In [123]:
text = '''"To thine own self be true,
And it must follow, as the night the day,
Thou canst not then be false to any man.
Farewell, my blessing season this in thee!'''

#PREDICT - 1ST WAY

In [124]:
pipe.__call__(text, top_k=3)

[{'label': 'Whittier John Greenleaf', 'score': 0.031478121876716614},
 {'label': 'Shakespeare William', 'score': 0.030998889356851578},
 {'label': 'Browning Robert', 'score': 0.030318431556224823}]

#PREDICT - 2ND WAY

In [125]:
inputs = predict_tokenizer(text, return_tensors="tf")
logits = predict_model(**inputs).logits

predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
predict_model.config.id2label[predicted_class_id]

'Whittier John Greenleaf'

#PREDICT - 3RD WAY

In [127]:
# Task 1: Create a dictionary with labels and scores
labels_scores_dict = {item['label']: item['score'] for sublist in pipe(text) for item in sublist}

# Task 2: Order the dictionary by score
sorted_dict = dict(sorted(labels_scores_dict.items(), key=lambda x: x[1], reverse=True))

# Task 3: Print the top 3 labels
count = 0
for label, score in sorted_dict.items():
    print(f"{label}: {score}")
    count += 1
    if count == 3:
        break

Whittier John Greenleaf: 0.031478121876716614
Shakespeare William: 0.030998889356851578
Browning Robert: 0.030318431556224823


#PREDICT PIPELINES FOR OTHER MODELS
##(weird cause same result)

In [128]:
model3 = TFAutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_model3')
tokenizer3 = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_tokenizer3')

pipe3 = TextClassificationPipeline(model=predict_model, tokenizer=predict_tokenizer, return_all_scores=True)

Some layers from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_model3 were not used when initializing TFDistilBertForSequenceClassification: ['dropout_195']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_model3 and are newly initialized: ['dropout_199']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

In [129]:
# Task 1: Create a dictionary with labels and scores
labels_scores_dict3 = {item['label']: item['score'] for sublist in pipe3(text) for item in sublist}

# Task 2: Order the dictionary by score
sorted_dict3 = dict(sorted(labels_scores_dict3.items(), key=lambda x: x[1], reverse=True))

# Task 3: Print the top 3 labels
count = 0
for label, score in sorted_dict3.items():
    print(f"{label}: {score}")
    count += 1
    if count == 3:
        break

Whittier John Greenleaf: 0.031478121876716614
Shakespeare William: 0.030998889356851578
Browning Robert: 0.030318431556224823


In [105]:
model4 = TFAutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_model4')
tokenizer4 = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_tokenizer4')

pipe4 = TextClassificationPipeline(model=predict_model, tokenizer=predict_tokenizer, return_all_scores=True)

Some layers from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_model4 were not used when initializing TFDistilBertForSequenceClassification: ['dropout_235']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_model4 and are newly initialized: ['dropout_119']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

In [106]:
# Task 1: Create a dictionary with labels and scores
labels_scores_dict4 = {item['label']: item['score'] for sublist in pipe4(text) for item in sublist}

# Task 2: Order the dictionary by score
sorted_dict4 = dict(sorted(labels_scores_dict4.items(), key=lambda x: x[1], reverse=True))

# Task 3: Print the top 3 labels
count = 0
for label, score in sorted_dict4.items():
    print(f"{label}: {score}")
    count += 1
    if count == 3:
        break

Whittier John Greenleaf: 0.03318120166659355
Shelley Percy Bysshe: 0.03157760947942734
Holmes Oliver Wendell: 0.03010241873562336


In [107]:
model5 = TFAutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_model5')
tokenizer5 = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_tokenizer5')

pipe5 = TextClassificationPipeline(model=predict_model, tokenizer=predict_tokenizer, return_all_scores=True)

Some layers from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_model5 were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/wagon/gutenBERT_model5 and are newly initialized: ['dropout_139']
You should probably TRAIN this model on a down-stream task to be able to use it for prediction

In [108]:
# Task 1: Create a dictionary with labels and scores
labels_scores_dict5 = {item['label']: item['score'] for sublist in pipe5(text) for item in sublist}

# Task 2: Order the dictionary by score
sorted_dict5 = dict(sorted(labels_scores_dict.items(), key=lambda x: x[1], reverse=True))

# Task 3: Print the top 3 labels
count = 0
for label, score in sorted_dict5.items():
    print(f"{label}: {score}")
    count += 1
    if count == 3:
        break

Whittier John Greenleaf: 0.03318120166659355
Shelley Percy Bysshe: 0.03157760947942734
Holmes Oliver Wendell: 0.03010241873562336
